In [3]:
import shutil
import random
from pathlib import Path

import cv2
import face_recognition

In [2]:
source_dir = Path("../data/LFW - People (Face Recognition)/lfw-funneled/lfw_funneled")
output_dir = Path("../data/LFW - People (Face Recognition)/lfw-funneled/lfw_funneled_faces")
output_dir.mkdir(parents=True, exist_ok=True)

In [3]:
#Extract only face from images
for img_path in source_dir.rglob("*.[jp][pn]g"):
    img = face_recognition.load_image_file(str(img_path))
    face_locations = face_recognition.face_locations(img)

    if not face_locations:
        print(f"Faces not found: {img_path}")
        continue    

    top, right, bottom, left = face_locations[0] 
    face_img = img[top:bottom, left:right]

    relative_path = img_path.relative_to(source_dir).parent
    save_dir = output_dir / relative_path
    save_dir.mkdir(parents=True, exist_ok=True)

    face_path = save_dir / img_path.name
    cv2.imwrite(str(face_path), cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR))

Faces not found: ../data/LFW - People (Face Recognition)/lfw-funneled/lfw_funneled/Jim_Abbott/Jim_Abbott_0001.jpg
Faces not found: ../data/LFW - People (Face Recognition)/lfw-funneled/lfw_funneled/Donald_Rumsfeld/Donald_Rumsfeld_0115.jpg
Faces not found: ../data/LFW - People (Face Recognition)/lfw-funneled/lfw_funneled/Tatiana_Gratcheva/Tatiana_Gratcheva_0001.jpg
Faces not found: ../data/LFW - People (Face Recognition)/lfw-funneled/lfw_funneled/John_Manley/John_Manley_0006.jpg
Faces not found: ../data/LFW - People (Face Recognition)/lfw-funneled/lfw_funneled/Jessica_Lynch/Jessica_Lynch_0001.jpg
Faces not found: ../data/LFW - People (Face Recognition)/lfw-funneled/lfw_funneled/Rob_Moore/Rob_Moore_0001.jpg
Faces not found: ../data/LFW - People (Face Recognition)/lfw-funneled/lfw_funneled/Jaouad_Gharib/Jaouad_Gharib_0001.jpg
Faces not found: ../data/LFW - People (Face Recognition)/lfw-funneled/lfw_funneled/Franz_Muentefering/Franz_Muentefering_0003.jpg
Faces not found: ../data/LFW - Peopl

In [4]:
# procces another dataset for group all faces by name (prefix)
faces_dir = Path("../data/Face Recognition Dataset/Faces/Faces")
output_dir = Path("../data/Face Recognition Dataset/Faces/oranaized_faces")
output_dir.mkdir(parents=True, exist_ok=True)

for img_path in faces_dir.rglob("*.[jp][pn]g"):
    name = "_".join(img_path.stem.split("_")[:-1])
    if not name:
        continue
    
    person_dir = output_dir / name
    person_dir.mkdir(parents=True, exist_ok=True)
    
    shutil.copy(str(img_path), person_dir / img_path.name)

In [ ]:
output_dir = Path("../data/dataset")
output_dir.mkdir(parents=True, exist_ok=True)

source_dirs = [
    Path("../data/LFW - People (Face Recognition)/lfw-funneled/lfw_funneled_faces"),
    Path("../data/Face Recognition Dataset/Faces/oranaized_faces"),
    Path("../data/casia-webface")
]

for src_dir in source_dirs:
    for subfolder in src_dir.iterdir():
        if subfolder.is_dir():
            dest = output_dir / subfolder.name
            shutil.copytree(subfolder, dest, dirs_exist_ok=True)

In [1]:
train_dir = output_dir / "train"
val_dir = output_dir / "val"
test_dir = output_dir / "test"

train_dir.mkdir(parents=True, exist_ok=True)
val_dir.mkdir(parents=True, exist_ok=True)
test_dir.mkdir(parents=True, exist_ok=True)

all_person_dirs = [p for p in output_dir.iterdir() if p.is_dir() and p.name not in ("train", "val", "test")]
random.seed(42)

for person_dir in all_person_dirs:
    images = list(person_dir.glob("*"))
    n = len(images)
    random.shuffle(images)

    n_train = max(int(n * 0.7), 1)
    n_val = max(int(n * 0.2), 1)
    n_test = max(n - n_train - n_val, 1)

    if n_train + n_val + n_test > n:
        n_train = n - n_val - n_test

    splits = {
        train_dir / person_dir.name: images[:n_train],
        val_dir / person_dir.name: images[n_train:n_train+n_val],
        test_dir / person_dir.name: images[n_train+n_val:n_train+n_val+n_test]
    }

    remaining = images[n_train+n_val+n_test:]
    if remaining:
        splits[train_dir / person_dir.name].extend(remaining)

    for dest_folder, split_images in splits.items():
        if not split_images:
            continue 
        dest_folder.mkdir(parents=True, exist_ok=True)
        for img_path in split_images:
            shutil.move(str(img_path), dest_folder / img_path.name)

    try:
        person_dir.rmdir()
    except OSError:
        pass


NameError: name 'output_dir' is not defined